# Estimating Work Tour Scheduling

This notebook illustrates how to re-estimate the mandatory tour scheduling component for ActivitySim.  This process 
includes running ActivitySim in estimation mode to read household travel survey files and write out
the estimation data bundles used in this notebook.  To review how to do so, please visit the other
notebooks in this directory.

# Load libraries

In [1]:
import os
import larch as lx
import pandas as pd

lx.versions()

JAX not found. Some functionality will be unavailable.


{'larch': '6.0.32',
 'sharrow': '2.13.0',
 'numpy': '1.26.4',
 'pandas': '1.5.3',
 'xarray': '2024.3.0',
 'numba': '0.60.0'}

For this demo, we will assume that you have already run ActivitySim in estimation
mode, and saved the required estimation data bundles (EDB's) to disk.  See
the [first notebook](./01_estimation_mode.ipynb) for details.  The following module
will run a script to set everything up if the example data is not already available.

In [2]:
from est_mode_setup import prepare

prepare()

EDB directory already populated.


PosixPath('test-estimation-data/activitysim-prototype-mtc-extended')

# Load data and prep model for estimation

In [3]:
modelname = "mandatory_tour_scheduling_work"

from activitysim.estimation.larch import component_model

model, data = component_model(
    modelname,
    edb_directory=f"output-est-mode/estimation_data_bundle/{modelname}/",
    return_data=True,
)

loading from output-est-mode/estimation_data_bundle/mandatory_tour_scheduling_work/tour_scheduling_work_coefficients.csv
loading from output-est-mode/estimation_data_bundle/mandatory_tour_scheduling_work/mandatory_tour_scheduling_work_SPEC.csv
loading from output-est-mode/estimation_data_bundle/mandatory_tour_scheduling_work/mandatory_tour_scheduling_work_alternatives_combined.parquet
loading from output-est-mode/estimation_data_bundle/mandatory_tour_scheduling_work/mandatory_tour_scheduling_work_choosers_combined.parquet


# Review data loaded from the EDB

The next (optional) step is to review the EDB, including the coefficients, utilities specification, and chooser and alternative data.

## Coefficients

In [4]:
data.coefficients

,value,constrain
coefficient_name,,
coef_dummy,1.000000,T
coef_free_flow_round_trip_auto_time_shift_effects_departure,-0.001140,F
coef_free_flow_round_trip_auto_time_shift_effects_duration,0.002210,F
coef_part_time_worker_departure_shift_effects,0.067360,F
coef_non_working_adult_duration_shift_effects,-0.120700,F
...,...,...
coef_duration_constants_9_hours,0.055706,F
coef_duration_constants_10_hours,0.000000,T
coef_duration_constants_11_hours,-0.347795,F


## Utility specification

In [5]:
data.spec

,Label,Description,Expression,Coefficient
0,util_free_flow_round_trip_auto_time_shift_effe...,Free-flow round trip auto time shift effects -...,roundtrip_auto_time_to_work * start,coef_free_flow_round_trip_auto_time_shift_effe...
1,util_free_flow_round_trip_auto_time_shift_effe...,Free-flow round trip auto time shift effects -...,roundtrip_auto_time_to_work * duration,coef_free_flow_round_trip_auto_time_shift_effe...
2,util_part_time_worker_departure_shift_effects,Part-time worker departure shift effects,(ptype == 2) * start,coef_part_time_worker_departure_shift_effects
3,util_non_working_adult_duration_shift_effects,Non-working adult duration shift effects,(ptype == 4) * duration,coef_non_working_adult_duration_shift_effects
4,util_university_student_departure_shift_effects,University student departure shift effects,(ptype == 3) * start,coef_university_student_departure_shift_effects
...,...,...,...,...
60,util_duration_constants_9_hours,Duration Constants -- 9 hours,duration == 9,coef_duration_constants_9_hours
61,util_duration_constants_10_hours,Duration Constants -- 10 hours,duration == 10,coef_duration_constants_10_hours
62,util_duration_constants_11_hours,Duration Constants -- 11 hours,duration == 11,coef_duration_constants_11_hours
63,util_duration_constants_12_to_13_hours,Duration Constants -- 12 to 13 hours,(duration > 11) & (duration < 14),coef_duration_constants_12_to_13_hours


## Chooser data

In [6]:
data.chooser_data

,tour_id,model_choice,override_choice,person_id,tour_type,tour_type_count,tour_type_num,tour_num,tour_count,tour_category,...,home_is_rural,mandatory_tour_frequency,is_worker,is_student,is_university,workplace_zone_id,school_zone_id,home_zone_id,start_previous,end_previous
0,2966594,7,96,72355,work,1,1,1,1,mandatory,...,False,work1,True,False,False,17,-1,55,5,5
1,2967783,50,30,72384,work,1,1,1,1,mandatory,...,False,work1,True,False,False,16,-1,59,5,5
2,2968726,64,29,72407,work,1,1,1,1,mandatory,...,False,work1,True,False,False,70,-1,59,5,5
3,2970858,31,80,72459,work,1,1,1,1,mandatory,...,False,work1,True,False,False,30,-1,61,5,5
4,2973728,61,115,72529,work,1,1,1,1,mandatory,...,False,work1,True,False,False,57,-1,69,5,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22632,309081532,32,10,7538573,work,1,1,1,1,mandatory,...,False,work1,True,False,False,135,-1,200,5,5
22633,309090634,48,79,7538795,work,1,1,1,1,mandatory,...,False,work1,True,False,False,1018,-1,1006,5,5
22634,309101950,69,78,7539071,work,1,1,1,1,mandatory,...,False,work1,True,False,False,1019,-1,1006,5,5
22635,309107362,139,53,7539203,work,1,1,1,1,mandatory,...,False,work1,True,False,False,1152,-1,1159,5,5


## Alternatives data

In [7]:
data.alt_values

,tour_id,start,end,duration,tdd,out_period,in_period,mode_choice_logsum,util_free_flow_round_trip_auto_time_shift_effects_departure,util_free_flow_round_trip_auto_time_shift_effects_duration,...,util_arrival_constants_late,util_duration_constants_0_to_2_hours,util_duration_constants_3_to_4_hours,util_duration_constants_5_to_6_hours,util_duration_constants_7_to_8_hours,util_duration_constants_9_hours,util_duration_constants_10_hours,util_duration_constants_11_hours,util_duration_constants_12_to_13_hours,util_duration_constants_14_to_18_hours
0,2966594,5,5,0,0,EA,EA,-0.897101,163.399994,0.000000,...,False,True,False,False,False,False,False,False,False,False
1,2966594,5,6,1,1,EA,AM,-1.333629,163.399994,32.680000,...,False,True,False,False,False,False,False,False,False,False
2,2966594,5,7,2,2,EA,AM,-1.333629,163.399994,65.360001,...,False,True,False,False,False,False,False,False,False,False
3,2966594,5,8,3,3,EA,AM,-1.333629,163.399994,98.040001,...,False,False,True,False,False,False,False,False,False,False
4,2966594,5,9,4,4,EA,AM,-1.333629,163.399994,130.720001,...,False,False,True,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4222354,309112036,21,22,1,185,EV,EV,-2.780379,1388.099976,66.099998,...,True,True,False,False,False,False,False,False,False,False
4222355,309112036,21,23,2,186,EV,EV,-2.780379,1388.099976,132.199997,...,True,True,False,False,False,False,False,False,False,False
4222356,309112036,22,22,0,187,EV,EV,-2.780379,1454.199951,0.000000,...,True,True,False,False,False,False,False,False,False,False
4222357,309112036,22,23,1,188,EV,EV,-2.780379,1454.199951,66.099998,...,True,True,False,False,False,False,False,False,False,False


# Estimate

With the model setup for estimation, the next step is to estimate the model coefficients.  Make sure to use a sufficiently large enough household sample and set of zones to avoid an over-specified model, which does not have a numerically stable likelihood maximizing solution.  Larch has a built-in estimation methods including BHHH, and also offers access to more advanced general purpose non-linear optimizers in the `scipy` package, including SLSQP, which allows for bounds and constraints on parameters.  BHHH is the default and typically runs faster, but does not follow constraints on parameters.

In [8]:
model.doctor(repair_ch_av="-")

problem: chosen_but_not_available has (83 issues)


(<larch.Model (MNL) "None">,
 ┣ chosen_but_not_available:    altid  n  example rows
 ┃                           0     23  1          7136
 ┃                           1     40  1          7611
 ┃                           2     41  2    2673, 3979
 ┃                           3     42  1         15549
 ┃                           4     44  2  10390, 17539
 ┃                           ..   ... ..           ...
 ┃                           78   175  1         15501
 ┃                           79   177  1         16777
 ┃                           80   178  1          1665
 ┃                           81   180  1          6817
 ┃                           82   184  1         10510
 ┃                           
 ┃                           [83 rows x 3 columns])

In [9]:
model.estimate(maxiter=900)

,value,best,initvalue,minimum,maximum,nullvalue,holdfast
param_name,,,,,,,
coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction,0.346120,0.346120,0.362700,-25.0,25.0,0.0,0
coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction,-0.097167,-0.097167,-0.101200,-25.0,25.0,0.0,0
coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction,0.103758,0.103758,0.177100,-25.0,25.0,0.0,0
coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction,-0.534895,-0.534895,-0.212300,-25.0,25.0,0.0,0
coef_arrival_constants_am_peak,-1.859817,-1.859817,-1.854521,-25.0,25.0,0.0,0
...,...,...,...,...,...,...,...
coef_subsequent_of_2plus_work_tours_duration_lt_8_hrs,2.609918,2.609918,2.582000,-25.0,25.0,0.0,0
coef_subsequent_tour_must_start_after_previous_tour_ends,-100.000000,-100.000000,-100.000000,-100.0,-100.0,0.0,1
coef_tours_by_student_duration_lt_8_hrs,2.619822,2.619822,2.582000,-25.0,25.0,0.0,0


/Users/jpn/Git/est-mode/larch/src/larch/model/jaxmodel.py:1156: PossibleOverspecification: Model is possibly over-specified (hessian is nearly singular).
  self.calculate_parameter_covariance()


┣                x: coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction               0.346120
┃                   coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction        -0.097167
┃                   coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction            0.103758
┃                   coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction     -0.534895
┃                   coef_arrival_constants_am_peak                                                          -1.859817
┃                                                                                                             ...    
┃                   coef_subsequent_of_2plus_work_tours_duration_lt_8_hrs                                    2.609918
┃                   coef_subsequent_tour_must_start_after_previous_tour_ends                              -100.000000
┃                   coef_tours_by_student_duration_lt_8_hrs                                                  2.619822
┃                   coef_tours_by_worker_duration_lt_8_hrs                                                   0.950422
┃                   coef_university_student_departure_shift_effects                                          0.045387
┃                   Length: 63, dtype: float64
┣          logloss: 3.971246101795334
┣        d_logloss: coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction            -2.476569e-05
┃                   coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction       2.401362e-05
┃                   coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction         -3.242733e-07
┃                   coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction    1.721606e-05
┃                   coef_arrival_constants_am_peak                                                         6.908123e-05
┃                                                                                                              ...     
┃                   coef_subsequent_of_2plus_work_tours_duration_lt_8_hrs                                  3.755607e-05
┃                   coef_subsequent_tour_must_start_after_previous_tour_ends                               0.000000e+00
┃                   coef_tours_by_student_duration_lt_8_hrs                                                5.453713e-05
┃                   coef_tours_by_worker_duration_lt_8_hrs                                                 5.453713e-05
┃                   coef_university_student_departure_shift_effects                                        5.979971e-05
┃                   Length: 63, dtype: float64
┣              nit: 55
┣             nfev: 69
┣             njev: 55
┣           status: 0
┣          message: 'Optimization terminated successfully'
┣          success: True
┣     elapsed_time: datetime.timedelta(seconds=40, microseconds=125911)
┣           method: 'slsqp'
┣          n_cases: 22637
┣ iteration_number: 55
┣          loglike: -89082.99255547294

### Estimated coefficients

In [10]:
model.parameter_summary()

,Value,Std Err,t Stat,Signif,Null Value,Constrained
Parameter,,,,,,
coef_adjacent_window_exists_after_this_arrival_hour_first_tour_interaction,0.346,0.844,0.41,,0.00,
coef_adjacent_window_exists_after_this_arrival_hour_second_plus_tour_interaction,-0.0972,0.215,-0.45,,0.00,
coef_adjacent_window_exists_before_this_departure_hour_first_tour_interaction,0.104,0.118,0.88,,0.00,
coef_adjacent_window_exists_before_this_departure_hour_second_plus_tour_interaction,-0.535,0.816,-0.66,,0.00,
coef_arrival_constants_am_peak,-1.86,0.145,-12.84,***,0.00,
coef_arrival_constants_early,0.00,0.00,NA,,0.00,fixed value
coef_arrival_constants_evening,0.109,0.0645,1.68,,0.00,
coef_arrival_constants_late,-0.888,0.0858,-10.35,***,0.00,
coef_arrival_constants_midday_1,-0.478,0.0618,-7.73,***,0.00,


# Output Estimation Results

In [11]:
from activitysim.estimation.larch import update_coefficients
result_dir = data.edb_directory/"estimated"
update_coefficients(
    model, data, result_dir,
    output_file=f"{modelname}_coefficients_revised.csv",
);

### Write the model estimation report, including coefficient t-statistic and log likelihood

In [12]:
model.to_xlsx(
    result_dir/f"{modelname}_model_estimation.xlsx", 
    data_statistics=False,
)

# Next Steps

The final step is to either manually or automatically copy the `*_coefficients_revised.csv` file to the configs folder, rename it to `*_coefficients.csv`, and run ActivitySim in simulation mode.

In [13]:
pd.read_csv(result_dir/f"{modelname}_coefficients_revised.csv")

,coefficient_name,value,constrain
0,coef_dummy,1.000000,T
1,coef_free_flow_round_trip_auto_time_shift_effe...,-0.001158,F
2,coef_free_flow_round_trip_auto_time_shift_effe...,0.002097,F
3,coef_part_time_worker_departure_shift_effects,0.058805,F
4,coef_non_working_adult_duration_shift_effects,-0.120700,F
...,...,...,...
59,coef_duration_constants_9_hours,0.041325,F
60,coef_duration_constants_10_hours,0.000000,T
61,coef_duration_constants_11_hours,-0.316543,F
62,coef_duration_constants_12_to_13_hours,-0.998700,F
